In [21]:
import wikipedia

pages = []
pages += wikipedia.search("science")
pages += wikipedia.search("religion")
pages += wikipedia.search("europe")
pages += wikipedia.search("asia")
pages += wikipedia.search("logic")
pages += wikipedia.search("language")
pages += wikipedia.random(10)
pages += wikipedia.random(10)
pages += wikipedia.random(10)
pages += wikipedia.random(10)
print(len(pages))

100


In [22]:
from transformers import AutoModelWithLMHead, AutoTokenizer
import random

model = AutoModelWithLMHead.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

def summarize(p, shuffle=False,min_length=0):
    summary = str()
    index = list(range(len(p)))
    if shuffle==True:
        random.shuffle(index)
    
    for i in index:
        if len(p[i])<100:
            continue
        ip = tokenizer.encode("summarize: " + p[i], return_tensors="pt", max_length=512, truncation=True)
        op = model.generate(ip, max_length=150, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
        op = tokenizer.decode(op[0])
        summary += op
        summary += " "
    summary=summary[:-1]
    return summary

p_model = AutoModelWithLMHead.from_pretrained("facebook/bart-large-cnn")
p_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

def p_summarize(p, shuffle=False,min_length=0):
    summary = str()
    index = list(range(len(p)))
    if shuffle==True:
        random.shuffle(index)
    
    for i in index:
        if len(p[i])<100:
            continue
        ip = p_tokenizer.encode("summarize: " + p[i], return_tensors="pt", max_length=1024, truncation=True)
        op = p_model.generate(ip, max_length=150, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
        op = p_tokenizer.decode(op[0])
        op = op[7:]
        summary += op
        summary += " "
    summary = summary[:-1]
    return summary

/Users/southdam/Desktop/Master-Project-Flashcard/Code/lib/python3.7/site-packages/transformers/modeling_auto.py:798: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
import language_tool_python
tool = language_tool_python.LanguageTool('en-US')

def get_error_rate(text):
    matches = tool.check(text)
    error = len(matches)
    # uncompleteness has more weight
    if text[-1]!="." and text[-1]!="!" and text[-1]!="?":
        error += 3
    text = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True)
    return error/len(text[0])

def get(n):
    para = wikipedia.summary(n)
    para = para.split('\n')
    return para

from eval import cos
c =cos()

self._url: http://127.0.0.1:8081/v2/


In [38]:
"""
for comparison
"""

names= ["Italy","China","Quasar"]
oo = []
ss = []
for i, name in tqdm(enumerate(names)):
    try:
        p = get(name)
        op= wikipedia.summary(name)
        oo.append(op)
    except wikipedia.DisambiguationError:
        continue

    summary = p_summarize(p,min_length=40)
    summary = [summary]
    bt = summarize(summary,min_length=80)
    ss.append(bt)
    bb = p_summarize(summary,min_length=80)
    ss.append(bb)
    t_summary = summarize(p,min_length=40)
    t_summary = [t_summary]
    tt = summarize(t_summary,min_length=80)
    ss.append(tt)

print(len(ss))
print(len(oo))

3it [03:51, 77.20s/it]

9
3


In [40]:
fake_simi = []
for wiki_art in oo:
    for sms in ss:
        fake_simi.append(c.eval(wiki_art,sms))
print(fake_simi)
print(sum(fake_simi)/len(fake_simi))

[0.7638484728730757, 0.8441363520983336, 0.7939456698964809, 0.248361692124814, 0.2626503510293116, 0.2787518961072287, 0.09632434892723558, 0.151872673608095, 0.0638286690643633, 0.3961785995747847, 0.30018253058466726, 0.32307617772751196, 0.7602597841180551, 0.755222522342109, 0.7045890231451821, 0.27910361012727514, 0.3366747499250633, 0.24207242034951595, 0.1465736430525994, 0.15218556986841916, 0.13088170102013566, 0.2746256076723271, 0.2806988960945043, 0.22528719329907387, 0.8362329284588995, 0.9014974752910011, 0.8645435903512334]
0.42272615365671473


In [35]:
"""
for 100 pages evaluation
"""
from tqdm import tqdm
error_rate_bt = []
error_rate_bb = []
error_rate_tt = []
simi_bt = []
simi_bb = []
simi_tt = []

for i, name in tqdm(enumerate(pages)):
    try:
        p = get(name)
        op= wikipedia.summary(name)
    except wikipedia.DisambiguationError:
        continue

    summary = p_summarize(p,min_length=40)
    summary = [summary]
    bt = summarize(summary,min_length=80)
    bb = p_summarize(summary,min_length=80)
    t_summary = summarize(p,min_length=40)
    t_summary = [t_summary]
    tt = summarize(t_summary,min_length=80)
    
    error_rate_bt.append(get_error_rate(bt))
    error_rate_bb.append(get_error_rate(bb))
    error_rate_tt.append(get_error_rate(tt))
    simi_bt.append(c.eval(op,bt))
    simi_bb.append(c.eval(op,bb))
    simi_tt.append(c.eval(op,tt))
    print(error_rate_bt)
    print(error_rate_bb)
    print(error_rate_tt)
    print(simi_bt)
    print(simi_bb)
    print(simi_tt)
    
print(error_rate_bt)
print(error_rate_bb)
print(error_rate_tt)
print(simi_bt)
print(simi_bb)
print(simi_tt)

1it [00:51, 51.29s/it]

[0.037037037037037035]
[0.02654867256637168]
[0.06666666666666667]
[0.5432880928916537]
[0.811578055423921]
[0.7426774203662289]


3it [01:39, 39.48s/it]

[0.037037037037037035, 0.11956521739130435]
[0.02654867256637168, 0.0707070707070707]
[0.06666666666666667, 0.12195121951219512]
[0.5432880928916537, 0.9477965895308955]
[0.811578055423921, 0.9432137409111842]
[0.7426774203662289, 0.8359256333487449]


4it [02:15, 38.50s/it]

...

[0.037037037037037035, 0.11956521739130435, 0.05194805194805195, 0.10989010989010989, 0.07216494845360824, 0.06666666666666667, 0.06060606060606061, 0.11494252873563218, 0.04878048780487805, 0.04878048780487805, 0.06593406593406594, 0.04878048780487805, 0.03614457831325301, 0.07792207792207792, 0.11842105263157894, 0.05333333333333334, 0.11827956989247312, 0.04878048780487805, 0.08163265306122448, 0.05747126436781609, 0.07526881720430108, 0.08641975308641975, 0.056179775280898875, 0.030612244897959183, 0.13333333333333333, 0.03571428571428571, 0.05, 0.06578947368421052, 0.08974358974358974, 0.037037037037037035, 0.021739130434782608, 0.075, 0.043478260869565216, 0.03, 0.038461538461538464, 0.03225806451612903, 0.12048192771084337, 0.10465116279069768, 0.04878048780487805, 0.056179775280898875, 0.039473684210526314, 0.12195121951219512, 0.1375, 0.07407407407407407, 0.06578947368421052, 0.07058823529411765, 0.05154639175257732, 0.058823529411764705, 0.02564102564102564, 0.037974683544303

62it [54:48, 49.56s/it]

[0.037037037037037035, 0.11956521739130435, 0.05194805194805195, 0.10989010989010989, 0.07216494845360824, 0.06666666666666667, 0.06060606060606061, 0.11494252873563218, 0.04878048780487805, 0.04878048780487805, 0.06593406593406594, 0.04878048780487805, 0.03614457831325301, 0.07792207792207792, 0.11842105263157894, 0.05333333333333334, 0.11827956989247312, 0.04878048780487805, 0.08163265306122448, 0.05747126436781609, 0.07526881720430108, 0.08641975308641975, 0.056179775280898875, 0.030612244897959183, 0.13333333333333333, 0.03571428571428571, 0.05, 0.06578947368421052, 0.08974358974358974, 0.037037037037037035, 0.021739130434782608, 0.075, 0.043478260869565216, 0.03, 0.038461538461538464, 0.03225806451612903, 0.12048192771084337, 0.10465116279069768, 0.04878048780487805, 0.056179775280898875, 0.039473684210526314, 0.12195121951219512, 0.1375, 0.07407407407407407, 0.06578947368421052, 0.07058823529411765, 0.05154639175257732, 0.058823529411764705, 0.02564102564102564, 0.037974683544303

62it [54:49, 53.06s/it]


IndexError: string index out of range

In [37]:
print(sum(error_rate_bt)/len(error_rate_bt))
print(sum(error_rate_bb)/len(error_rate_bb))
print(sum(error_rate_tt)/len(error_rate_tt))
print(sum(simi_bt)/len(simi_bt))
print(sum(simi_bb)/len(simi_bb))
print(sum(simi_tt)/len(simi_tt))

0.06742919593914502
0.028683787661541185
0.07193084300388397
0.817700766706108
0.8424491404527759
0.8196131275985171


In [3]:
import wikipedia
p = wikipedia.summary("Italy")
print(p)

Italy (Italian: Italia [iˈtaːlja] (listen)), officially the Italian Republic (Italian: Repubblica Italiana [reˈpubblika itaˈljaːna]),  is a sovereign country consisting of a peninsula delimited by the Alps and surrounded by several islands. Italy is located in south-central Europe, and is considered part of western Europe. A unitary parliamentary republic with Rome as its capital, the country covers a total area of 301,340 km2 (116,350 sq mi) and shares land borders with France, Switzerland, Austria, Slovenia, and the enclaved microstates of Vatican City and San Marino. Italy has a territorial exclave in Switzerland (Campione) and a maritime exclave in Tunisian waters (Lampedusa). With around 60 million inhabitants, Italy is the third-most populous member state of the European Union.
Due to its central geographic location in Southern Europe and the Mediterranean, Italy has historically been home to myriad peoples and cultures. In addition to the various ancient peoples dispersed throug

In [6]:
from eval import cos

evaluator=cos()

summary_b_t = "Italy is a unitary parliamentary republic with its capital in Rome. with around 60 million inhabitants, it is the third-most populous member state of the eu. it shares land borders with France, Switzerland, Austria, Slovenia, and Vatican City and San Marino. it has been home to numerous ancient peoples and cultures. the most predominant are the Indo-European Italic peoples."
summary_b_b = "With around 60 million inhabitants, Italy is the third-most populous member state of the European Union. The country covers a total area of 301,340 km2 (116,350 sq mi) and shares land borders with France, Switzerland, Austria, Slovenia, and Vatican City and San Marino. Italy has been home to numerous ancient peoples and cultures. The most predominant are the Indo-European Italic peoples who gave the peninsula its name."
summary_t_t = "Italy is a unitary parliamentary republic with its capital in Rome. it covers a total area of 301,340 km2 (116,350 sq mi) and shares land borders with France, Switzerland, Austria, Slovenia. with around 60 million inhabitants, it is the third-most populous member state of the european union. Italy is considered one of the world's most culturally and economically advanced countries"
summary_t_b = summary_t_t
print(evaluator.eval(p,summary_b_t))
print(evaluator.eval(p,summary_b_b))
print(evaluator.eval(p,summary_t_t))
print(evaluator.eval(p,summary_t_b))

0.7342861822168165
0.8310094443281635
0.7743800208018697
0.7743800208018697


In [8]:
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer('bert-large-nli-stsb-mean-tokens')

c =cos()
text = c.get_st(p)
bt = c.get_st(summary_b_t)
bb = c.get_st(summary_b_b)
tt = c.get_st(summary_t_t)
tb = c.get_st(summary_t_b)
sentence_embeddings = sentence_model.encode(text)
bt_embeddings = sentence_model.encode(bt)
bb_embeddings = sentence_model.encode(bb)
tt_embeddings = sentence_model.encode(tt)
tb_embeddings = sentence_model.encode(tb)
"""
for overall similarity
"""
self_simi=[]
bt_simi = []
bb_simi = []
tt_simi = []
tb_simi = []
for t in sentence_embeddings:
    for s1 in bt_embeddings:
        bt_simi.append(c.cosine(t,s1))
    for s2 in bb_embeddings:
        bb_simi.append(c.cosine(t,s2))
    for s3 in tt_embeddings:
        tt_simi.append(c.cosine(t,s3))
    for s4 in tb_embeddings:
        tb_simi.append(c.cosine(t,s4))
    for nt in sentence_embeddings:
        self_simi.append(c.cosine(t,nt))
print(bt_simi)
print(bb_simi)
print(tt_simi)
print(tb_simi)
print(self_simi)
print(sum(bt_simi)/len(bt_simi))
print(sum(bb_simi)/len(bb_simi))
print(sum(tt_simi)/len(tt_simi))
print(sum(tb_simi)/len(tb_simi))
print(sum(self_simi)/len(self_simi))

[0.52067244, 0.07913756, 0.55282164, 0.017449277, 0.31205475, 0.47789025, 0.19401245, 0.33861348, 0.027486715, 0.39253464, 0.59468454, 0.33430123, 0.6785839, 0.20795026, 0.31794667, 0.54122734, 0.17761607, 0.4890242, 0.14843546, 0.22586921, 0.50615907, 0.82384235, 0.20343114, 0.16251901, 0.4114089, 0.4301706, 0.2768577, 0.3876315, 0.5008476, 0.46997485, 0.35740155, 0.21153459, 0.345458, 0.50926256, 0.5908052, 0.5669373, 0.2164445, 0.27501857, 0.26926073, 0.34155303, 0.27390212, 0.18849157, 0.3540198, 0.37391007, 0.38555145, 0.4907208, 0.21553868, 0.23931122, 0.3112972, 0.44526276, 0.56488174, 0.1889674, 0.37583578, 0.29093668, 0.42900547, 0.24806678, 0.09639983, 0.15000556, 0.19836794, 0.40480778, 0.25288463, 0.27003732, 0.3441305, 0.28445777, 0.30194345, 0.33258706, 0.18855245, 0.27127835, 0.29170564, 0.29338402, 0.41413078, 0.21157885, 0.2886934, 0.3727982, 0.43030503, 0.37012115, 0.21793364, 0.3004222, 0.22478518, 0.3218279, 0.25939748, 0.11357387, 0.0075257053, 0.034418553, 0.29235

In [28]:
"""
for paragraph vector
"""
def new_vector(vectors):
    for i,v in enumerate(vectors):
        vectors[i]=np.array(v)
    vector = np.zeros(1024)
    for __, v in enumerate(vectors):
        vector += v
    return vector

se=new_vector(sentence_embeddings)
ste=new_vector(st_embeddings)
ne=new_vector(neu_embeddings)
print(se)
print(ste)
print(ne)
print(cosine(se,se))
print(cosine(se,ste))
print(cosine(se,ne))

[-2.6610069  -0.66507934 -2.00386417 ...  2.0526337   0.81142529
  2.76544661]
[-2.27682245 -1.46035932 -1.35502264 ... -0.08776828  0.54368304
  0.47381221]
[ 0.58004402 -1.46709879 -1.25582227 ... -0.34789667  2.0698542
  0.40905179]
1.0
0.887168104204225
0.07189953337818449
